In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
import os
import cv2

Using TensorFlow backend.


In [ ]:
### Organizing classes into training, validation, and test
print("Organizing datasets")
masked_imgs = os.listdir('datasets/compiled/with_mask copy')
no_mask_imgs = os.listdir('datasets/compiled/without_mask copy')

i = 0
for img in masked_imgs:
	if i < int(0.64*len(masked_imgs)):
		os.rename('datasets/compiled/with_mask copy/' + img, 'datasets/compiled/train/mask/' + img)
		i += 1
	elif i < int(0.8*len(masked_imgs)):
		os.rename('datasets/compiled/with_mask copy/' + img, 'datasets/compiled/validation/mask/' + img)
		i += 1
	else:
		os.rename('datasets/compiled/with_mask copy/' + img, 'datasets/compiled/test/mask/' + img)
		i += 1

j = 0
for img in no_mask_imgs:
	if j < int(0.64*len(no_mask_imgs)):
		os.rename('datasets/compiled/without_mask copy/' + img,'datasets/compiled/train/no-mask/' + img)
		j += 1
	elif j < int(0.8*len(no_mask_imgs)):
		os.rename('datasets/compiled/without_mask copy/' + img, 'datasets/compiled/validation/no-mask/' + img)
		j += 1
	else:
		os.rename('datasets/compiled/without_mask copy/' + img, 'datasets/compiled/test/no-mask/' + img)
		j += 1

print("Images reorganized")

In [10]:
epochs = 1
batch_size = 32

In [3]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255)

validation_datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255)

test_datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        batch_size=batch_size,
		directory='datasets/compiled/train/',
        target_size=(200, 300), 
        classes = ['mask','no-mask'],
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        batch_size=batch_size,
        directory='datasets/compiled/validation/',
        target_size=(200, 300), 
        classes = ['mask','no-mask'],
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
		batch_size=batch_size,
        directory='datasets/compiled/test/',
        target_size=(200, 300), 
        classes = ['mask','no-mask'],
        class_mode='categorical')

Found 2206 images belonging to 2 classes.
Found 552 images belonging to 2 classes.
Found 1075 images belonging to 2 classes.


In [5]:
### Pre-trained Model (ResNet50 trained on Imagenet)
model = tf.keras.applications.ResNet50(include_top=False,weights='imagenet')

In [6]:
# Transfer Learning
for i in model.layers:
  i.trainable = False

In [7]:
global_avg = tf.keras.layers.GlobalAveragePooling2D()(model.output)
drop_out = tf.keras.layers.Dropout(0.4)(global_avg)
out = tf.keras.layers.Dense(2,activation='sigmoid')(drop_out)
resnet = tf.keras.Model(inputs=[model.input],outputs=[out])

In [8]:
resnet.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

In [11]:
history = resnet.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

69/69 [==============================] - 204s 3s/step - loss: 0.7191 - accuracy: 0.5349 - val_loss: 0.6874 - val_accuracy: 0.5562


In [12]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [14]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=resnet.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'